- `review_date` and `claim_date` is not always yyyy-mm-dd
    - Contains yyyy-mm-ddThh:mm:ss and None

In [2]:
from helper import download_dataset, download_article

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
DATASET_FP = "./WatClaimCheck_dataset" # CHANGE TO MATCH LOCAL
train_pd_df, valid_pd_df, test_pd_df = download_dataset(DATASET_FP)

In [4]:
def explode_dictionary(pd_df: pd.DataFrame, field: str) -> pd.DataFrame:
    '''Explodes a dictionary within a column as multiple columns and then drops the parent field'''
    return pd.concat([pd_df.drop(field, axis=1), pd_df[field].apply(pd.Series)], axis=1)

def clean_pd_df(pd_df: pd.DataFrame) -> pd.DataFrame:
    '''Extracts field from dataframe and removes id'''
    pd_df = explode_dictionary(pd_df, 'metadata')
    pd_df = explode_dictionary(pd_df, 'label')

    pd_df = pd_df.drop(columns=['id'])
    
    return pd_df

In [5]:
flatten_train_pd_df = clean_pd_df(train_pd_df)
flatten_valid_pd_df = clean_pd_df(valid_pd_df)
flatten_test_pd_df = clean_pd_df(test_pd_df)

flatten_train_pd_df.dtypes

claimant            object
claim               object
claim_date          object
review_date         object
premise_articles    object
reviewer_name       object
reviewer_site       object
review_url          object
rating               int64
original_rating     object
review_article      object
dtype: object

## Join dataframe with articles

In [7]:
def get_articles(premise_articles_dict):
    json_list = list(premise_articles_dict.values())
    json_list_content = [download_article(DATASET_FP, i) for i in json_list]
    return json_list

flatten_train_pd_df['premise_articles_content'] = flatten_train_pd_df['premise_articles'].apply(get_articles)
flatten_train_pd_df

FileNotFoundError: [Errno 2] No such file or directory: './WatClaimCheck_dataset/articles/1_1.json'